In [54]:
from ultralytics import YOLO
import cv2
import json
import os
from pathlib import Path
import numpy as np
from symspellpy import SymSpell, Verbosity
import re

# Настройка путей
model_path = "day4/runs/detect/train/weights/best.pt"  # Путь к модели YOLO
images_dir = "301"  # Папка с изображениями
json_dir = "day1/school"  # Папка с JSON-файлами
save_folder = "result"  # Папка для результатов
dictionary_path = "day4/russian.txt"  # Путь к словарю SymSpell
numbers = {'0': '22', '1': '23', '2': '24', '3': '25', '4': '26', '5': '27', '6': '28', 'none': 'none'}

In [55]:
# Функция для проверки, является ли строка словом
def is_word(s: str) -> bool:
    # Проверяет, что строка содержит только буквы кириллицы или дефисы и имеет хотя бы одну букву
    return bool(re.match(r'^[а-яА-Я\-]*[а-яА-Я][а-яА-Я\-]*$', s))

# Функция для объединения слов с дефисами
def merge_words(words: list[str]) -> list[str]:
    # Возвращает новый список, объединяя слова, где первое заканчивается на дефис
    result = []
    i = 0
    while i < len(words):
        if not is_word(words[i]):
            result.append(words[i])
            i += 1
            continue
        if words[i].endswith('-') and i + 1 < len(words) and is_word(words[i + 1]):
            merged_word = words[i][:-1] + words[i + 1]
            if is_word(merged_word):
                result.append(merged_word)
                i += 2  # Пропускаем два слова
            else:
                result.append(words[i])
                i += 1
        else:
            result.append(words[i])
            i += 1
    return result

# Функция для поиска исправленного слова с помощью SymSpell
def finder_symspell(word: str, sym_spell: SymSpell, max_edit_distance: int = 3) -> str:
    word_lower = word.lower()
    suggestions = sym_spell.lookup(word_lower, Verbosity.TOP, max_edit_distance=0)
    if suggestions:
        return word
    suggestions = sym_spell.lookup(word_lower, Verbosity.TOP, max_edit_distance)
    if suggestions:
        return suggestions[0].term
    return word_lower

# Функция для инициализации SymSpell
def init_symspell(path: str, max_edit_distance: int = 3) -> SymSpell:
    sym_spell = SymSpell(max_dictionary_edit_distance=max_edit_distance)
    try:
        with open(path, 'r', encoding='utf-8') as f:
            for word in f:
                word = word.rstrip()
                if is_word(word):
                    sym_spell.create_dictionary_entry(word, 1)
    except FileNotFoundError:
        print(f"Файл словаря {path} не найден")
        raise
    return sym_spell

In [56]:
# Функция для вычисления IoU
def calculate_iou(box1, box2):
    x1_min, y1_min, x1_max, y1_max = box1  # JSON box
    x2_min, y2_min, x2_max, y2_max = box2  # YOLO box
    
    x_left = max(x1_min, x2_min)
    y_top = max(y1_min, y2_min)
    x_right = min(x1_max, x2_max)
    y_bottom = min(y1_max, y2_max)
    
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    
    intersection = (x_right - x_left) * (y_bottom - y_top)
    area1 = (x1_max - x1_min) * (y1_max - y1_min)  # JSON box area
    area2 = (x2_max - x2_min) * (y2_max - y2_min)  # YOLO box area
    union = area1 + area2 - intersection
    
    iou = intersection / union if union > 0 else 0.0
    return iou

# Функция для загрузки JSON
def load_json(json_path):
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        return data
    except Exception as e:
        print(f"Ошибка при загрузке {json_path}: {e}")
        return None

# Функция для извлечения и исправления текста из JSON
def extract_text_from_json(json_data, sym_spell):
    texts = []
    raw_words = []

    if not json_data or not isinstance(json_data, dict):
        print(f"JSON некорректен или пуст: {json_data}")
        return texts

    data = json_data.get("data")
    if not data or not isinstance(data, dict):
        print(f"Ключ 'data' отсутствует или некорректен: {json_data}")
        return texts

    blocks = data.get("blocks")
    if not blocks or not isinstance(blocks, list):
        print(f"Ключ 'blocks' отсутствует или некорректен: {json_data}")
        return texts

    # Извлечение слов и их координат
    for block in blocks:
        boxes = block.get("boxes")
        if not boxes or not isinstance(boxes, list):
            continue

        for box in boxes:
            languages = box.get("languages")
            if not languages or not isinstance(languages, list):
                continue

            for lang in languages:
                texts_list = lang.get("texts")
                if not texts_list or not isinstance(texts_list, list):
                    continue

                for text in texts_list:
                    words = text.get("words")
                    if not words or not isinstance(words, list):
                        continue

                    for word in words:
                        word_text = word.get("word", "")
                        x = word.get("x", 0)
                        y = word.get("y", 0)
                        w = word.get("w", 0)
                        h = word.get("h", 0)
                        raw_words.append(word_text)
                        texts.append({
                            "text": word_text,
                            "bbox": [x, y, x + w, y + h]
                        })

    if not texts:
        print(f"Не найдены тексты в JSON: {json_data}")
        return texts

    # Объединение слов с дефисами
    merged_words = merge_words(raw_words)

    # Исправление слов с помощью SymSpell
    corrected_words = [finder_symspell(word, sym_spell) for word in merged_words]

    # Обновление текста в списке texts
    corrected_index = 0
    for i, text_item in enumerate(texts):
        if corrected_index < len(corrected_words):
            texts[i]["text"] = corrected_words[corrected_index]
            if (
                i + 1 < len(raw_words)
                and raw_words[i].endswith("-")
                and is_word(raw_words[i + 1])
            ):
                corrected_index += 1  # Пропустить объединённое слово
            corrected_index += 1

    return texts


In [57]:
# Инициализация SymSpell
try:
    sym_spell = init_symspell(dictionary_path)
except FileNotFoundError as e:
    print(f"Не удалось инициализировать SymSpell: {e}")
    raise

In [ ]:
os.makedirs(save_folder, exist_ok=True)
model = YOLO(model_path)
excluded_classes = [7, 8]  # Классы для исключения

# Проход по JSON-файлам
for json_file in Path(json_dir).glob("*res.json"):
    json_name = json_file.stem  # Имя JSON без расширения (например, 1119273396_03__1_res)
    img_name = json_name.split("__")[0]  # Извлекаем префикс (например, 1119273396_03)
    img_file = Path(images_dir) / f"{img_name}.png"
    output_file = Path(save_folder) / f"{json_name}.txt"  # Файл результата
    
    if not img_file.exists():
        print(f"Изображение для {json_name} не найдено: {img_file}")
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(f"Изображение {img_name}.png не найдено\n")
        continue
    
    # Чтение JSON
    json_data = load_json(json_file)
    if json_data is None:
        print(f"Пропуск {json_name} из-за ошибки загрузки JSON")
        continue
    
    json_texts = extract_text_from_json(json_data, sym_spell)
    if not json_texts:
        print(f"В {json_name} не найдены тексты")
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write("Нет текстов в JSON\n")
        continue
    
    # Получение предсказаний YOLO для изображения
    try:
        results = model.predict(source=str(img_file), imgsz=640, save=False, conf=0.1)
        result = results[0]  # Предполагаем, что одно изображение
    except Exception as e:
        print(f"Ошибка YOLO для {img_name}: {e}")
        with open(output_file, 'w', encoding='utf-8') as f:
            for json_text in json_texts:
                f.write(f"{numbers['none']}:\n{json_text['text']}\n\n")
            f.write(f"Ошибка YOLO: {e}\n")
        continue
    
    # Получение предсказаний
    bboxes = result.boxes.xyxy.cpu().numpy()  # [x_min, y_min, x_max, y_max]
    classes = result.boxes.cls.cpu().numpy()  # Метки классов
    confidences = result.boxes.conf.cpu().numpy()  # Вероятности
    
    # Фильтрация исключённых классов
    valid_indices = [i for i, cls in enumerate(classes) if cls not in excluded_classes]
    bboxes = bboxes[valid_indices]
    classes = classes[valid_indices]
    confidences = confidences[valid_indices]
    
    # Сбор текстов по заданиям
    task_texts = {num: [] for num in numbers.values()}  # Словарь для текстов по заданиям
    for json_text in json_texts:
        json_bbox = json_text["bbox"]
        best_iou = 0.0
        best_conf = 0.0
        best_number = "none"
        
        for i, bbox in enumerate(bboxes):
            iou = calculate_iou(json_bbox, bbox)
            if iou > best_iou or (iou == best_iou and confidences[i] > best_conf):  # Выбираем бокс с максимальным IoU или максимальной уверенностью
                best_iou = iou
                best_conf = confidences[i]
                best_number = str(int(classes[i]))
        
        task_texts[numbers[best_number]].append(json_text["text"])
    
    # Формирование вывода
    matches = []
    for task_num in sorted(task_texts.keys()):
        if task_texts[task_num]:  # Пропускаем пустые задания
            matches.append(f"{task_num}:\n{' '.join(task_texts[task_num])}\n")
    
    # Сохранение результатов в файл
    try:
        with open(output_file, 'w', encoding='utf-8') as f:
            for line in matches:
                f.write(line + "\n")
        print(f"Результаты для {json_name} сохранены в {output_file}")
    except Exception as e:
        print(f"Ошибка записи в файл {output_file}: {e}")
        continue

In [19]:
print(model.names)

{0: '22', 1: '23', 2: '24', 3: '25', 4: '26', 5: '27', 6: '28', 7: 'номер', 8: 'текст'}


In [61]:
from collections import defaultdict


# Настройка путей
result_dir = "result"  # Папка с исходными файлами
output_dir = "combined_results"  # Папка для объединённых результатов
os.makedirs(output_dir, exist_ok=True)  # Создаём папку, если не существует

# Функция для извлечения префикса (номера варианта)
def extract_prefix(file_name):
    # Ищем префикс до '_XX', где XX — номер страницы (например, '_02')
    match = re.match(r'^(.*?)(_\d{2})__.*$', file_name)
    if match:
        return match.group(1)  # Возвращаем префикс, например, '1019641987'
    return file_name  # Если формат не соответствует, возвращаем имя как есть

# Функция для чтения текста из файла
def read_task_texts(file_path):
    task_texts = defaultdict(list)  # Словарь для текстов по заданиям
    current_task = None
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            for line in lines:
                line = line.strip()
                if not line:
                    continue
                if line.endswith(':'):  # Начало нового задания
                    current_task = line[:-1]  # Убираем двоеточие
                elif current_task and line not in ["Нет текстов в JSON", "Ошибка YOLO"]:  # Игнорируем сообщения об ошибках
                    task_texts[current_task].append(line)
    except Exception as e:
        print(f"Ошибка чтения файла {file_path}: {e}")
    return task_texts

# Сбор всех файлов и группировка по префиксу
prefix_files = defaultdict(list)
for file_path in Path(result_dir).glob("*.txt"):
    file_name = file_path.stem
    prefix = extract_prefix(file_name)  # Извлекаем префикс, например, '1019641987'
    prefix_files[prefix].append(file_path)

# Обработка каждой группы файлов
for prefix, files in prefix_files.items():
    combined_texts = defaultdict(list)
    
    # Чтение текстов из всех файлов с одинаковым префиксом
    for file_path in files:
        task_texts = read_task_texts(file_path)
        for task_num, texts in task_texts.items():
            combined_texts[task_num].extend(texts)
    
    # Формирование объединённого вывода
    output_lines = []
    for task_num in sorted(combined_texts.keys()):  # Сортировка заданий
        if combined_texts[task_num]:  # Пропускаем пустые задания
            # Объединяем тексты в одну строку
            combined_text = " ".join(combined_texts[task_num])
            output_lines.append(f"{task_num}:\n{combined_text}\n")
    
    # Сохранение результата
    output_file = Path(output_dir) / f"{prefix}_combined.txt"
    try:
        with open(output_file, 'w', encoding='utf-8') as f:
            for line in output_lines:
                f.write(line + "\n")
        print(f"Объединённые результаты для {prefix} сохранены в {output_file}")
    except Exception as e:
        print(f"Ошибка записи в файл {output_file}: {e}")

Объединённые результаты для 2721922393 сохранены в combined_results/2721922393_combined.txt
Объединённые результаты для 2420005454 сохранены в combined_results/2420005454_combined.txt
Объединённые результаты для 2719810076 сохранены в combined_results/2719810076_combined.txt
Объединённые результаты для 2820748417 сохранены в combined_results/2820748417_combined.txt
Объединённые результаты для 2220115934 сохранены в combined_results/2220115934_combined.txt
Объединённые результаты для 2921008493 сохранены в combined_results/2921008493_combined.txt
Объединённые результаты для 2220198818 сохранены в combined_results/2220198818_combined.txt
Объединённые результаты для 2220192253 сохранены в combined_results/2220192253_combined.txt
Объединённые результаты для 2820731400 сохранены в combined_results/2820731400_combined.txt
Объединённые результаты для 1120331958 сохранены в combined_results/1120331958_combined.txt
Объединённые результаты для 2819688928 сохранены в combined_results/2819688928_c